In [1]:
#Import the package to your work environment
import fingertips_py as ftp
import pandas as pd
import altair as alt

In [2]:
# Define the Indicator ID: Depression incidence
depression = 848 

# Fetch the data for the specified indicator at all available geographies
depression_data = ftp.get_data_for_indicator_at_all_available_geographies(depression)

# Define the Indicator ID: Deprivation score
deprivation_score = 93553

# Fetch the data for the specified indicator at all available geographies
deprevitation_data = ftp.get_data_for_indicator_at_all_available_geographies(deprivation_score)

# Define the Indicator ID: Employment Deprivation score
employment_deprivation_score = 92628

# Fetch the data for the specified indicator at all available geographies
emp_deprivation_data = ftp.get_data_for_indicator_at_all_available_geographies(employment_deprivation_score)

# Define the Indicator ID: Crime Deprivation score
crime_deprivation_score = 92635

# Fetch the data for the specified indicator at all available geographies
crime_deprivation_data = ftp.get_data_for_indicator_at_all_available_geographies(crime_deprivation_score)



c:\Bruno\Python\Lib\site-packages\fingertips_py\retrieve_data.py:46: DtypeWarning: Columns (9,10,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(base_url + populated_url)


In [3]:
# Inspect columns
depression_data.columns
depression_data["Area Type"].unique()
depression_data_counties = depression_data[depression_data["Area Type"]=="Districts & UAs (from Apr 2023)"]

depression_data_counties["Area Type"].unique()

depression_data_counties["Time period"].unique()

depres = depression_data_counties[depression_data_counties["Time period"]=="2022/23"]

depres["Time period"].unique()

array(['2022/23'], dtype=object)

In [5]:
# Inspect columns
deprevitation_data.columns
deprevitation_data["Area Type"].unique()

deprivation_data_counties = deprevitation_data[deprevitation_data["Area Type"]=="Districts & UAs (from Apr 2023)"]

deprivation_data_counties["Area Type"].unique()

deprivation_data_counties["Time period"].unique()

deprivation = deprivation_data_counties[deprivation_data_counties["Time period"]==2019]

deprivation["Time period"].unique()


array([2019], dtype=object)

In [6]:
# Inspect columns
emp_deprivation_data.columns
emp_deprivation_data["Area Type"].unique()

emp_deprivation_data_counties = emp_deprivation_data[emp_deprivation_data["Area Type"]=="Districts & UAs (from Apr 2023)"]

emp_deprivation_data_counties["Area Type"].unique()

emp_deprivation_data_counties["Time period"].unique()

employment = emp_deprivation_data_counties[emp_deprivation_data_counties["Time period"]==2019]

employment["Time period"].unique()

array([2019], dtype=int64)

In [7]:
# Inspect columns
crime_deprivation_data.columns
crime_deprivation_data["Area Type"].unique()

crime_deprivation_data_counties = crime_deprivation_data[crime_deprivation_data["Area Type"]=="Districts & UAs (from Apr 2023)"]

crime_deprivation_data_counties["Area Type"].unique()

crime_deprivation_data_counties["Time period"].unique()

crime = crime_deprivation_data_counties[crime_deprivation_data_counties["Time period"]==2019]

crime["Area Type"].unique()

crime["Time period"].unique()


array([2019], dtype=int64)

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

    # Merge depression with deprivation
merged_depression_deprivation = depres.merge(deprivation, on="Area Code", suffixes=("_depression", "_deprivation"))

    # Merge depression with employment deprivation
merged_depression_employment = depres.merge(employment, on="Area Code", suffixes=("_depression", "_employment"))

    # Merge depression with crime deprivation
merged_depression_crime = depres.merge(crime, on="Area Code", suffixes=("_depression", "_crime"))


In [24]:
import altair as alt

# For Deprivation Scatter Plot
data_deprivation = merged_depression_deprivation[[
    "Value_depression",  # Depression Prevalence
    "Value_deprivation",  # Deprivation Score
    "Area Name_deprivation"  # District Name
]].rename(
    columns={
        "Value_depression": "Depression Prevalence",
        "Value_deprivation": "Deprivation Score",
        "Area Name_deprivation": "District Name"
    }
)

# Create dual sliders for Deprivation Score
x_min_slider = alt.param(
    name='Deprivation_Score_Min',
    bind=alt.binding_range(
        min=data_deprivation['Deprivation Score'].min(), 
        max=data_deprivation['Deprivation Score'].max(), 
        step=1
    ),
    value=data_deprivation['Deprivation Score'].min()  # Default value
)

x_max_slider = alt.param(
    name='Deprivation_Score_Max',
    bind=alt.binding_range(
        min=data_deprivation['Deprivation Score'].min(), 
        max=data_deprivation['Deprivation Score'].max(), 
        step=1
    ),
    value=data_deprivation['Deprivation Score'].max()  # Default value
)

# Create dual sliders for Depression Prevalence
y_min_slider = alt.param(
    name='Depression_Prevalence_Min',
    bind=alt.binding_range(
        min=data_deprivation['Depression Prevalence'].min(), 
        max=data_deprivation['Depression Prevalence'].max(), 
        step=1
    ),
    value=data_deprivation['Depression Prevalence'].min()  # Default value
)

y_max_slider = alt.param(
    name='Depression_Prevalence_Max',
    bind=alt.binding_range(
        min=data_deprivation['Depression Prevalence'].min(), 
        max=data_deprivation['Depression Prevalence'].max(), 
        step=1
    ),
    value=data_deprivation['Depression Prevalence'].max()  # Default value
)

# Base chart for scatter plot and regression line
base = alt.Chart(data_deprivation).transform_filter(
    f"datum['Deprivation Score'] >= {x_min_slider.name} && " +
    f"datum['Deprivation Score'] <= {x_max_slider.name} && " +
    f"datum['Depression Prevalence'] >= {y_min_slider.name} && " +
    f"datum['Depression Prevalence'] <= {y_max_slider.name}"
).add_params(
    x_min_slider, x_max_slider, y_min_slider, y_max_slider
)

# Scatter plot with filtering
scatter_with_regression = base.mark_circle(size=80, color ="blue").encode(
    x=alt.X('Deprivation Score:Q', title='Deprivation Score (IMD 2019)', scale=alt.Scale(zero=False)),
    y=alt.Y('Depression Prevalence:Q', title='Depression Prevalence (%)', scale=alt.Scale(zero=False)), 
    tooltip=[
        alt.Tooltip('District Name:N', title='District Name'),
        alt.Tooltip('Deprivation Score:Q', title='Deprivation Score (IMD 2019)'),
        alt.Tooltip('Depression Prevalence:Q', title='Depression Prevalence (%)')
    ]
).properties(
    width=1000,
    height=600
)

# Regression line that dynamically adjusts to filtered data
regression_line = base.transform_regression(
    'Deprivation Score', 'Depression Prevalence', method='linear'
).mark_line(color='red').encode(
    x='Deprivation Score:Q',
    y='Depression Prevalence:Q'
)

# Combine scatter plot and regression line
final_chart = (scatter_with_regression + regression_line).properties(
    title={
        "text": "Depression Prevalence (2023) vs Deprivation Score (2019)",
        "subtitle": "Use the sliders to adjust the range and explore the data",
        "fontSize": 20,
        "subtitleFontSize":14,
        "anchor": "start"
    }
).configure_title(
    fontSize=18,
    anchor='start',
    color='black'
)

# Display the chart
final_chart.interactive()

output_file = 'C:\Bruno\LSE\PP434 - Automated Data Visualisation\Project\Chart_3_scatter_with_deprivation.json'
final_chart.save(output_file, format='json')


In [25]:
# Prepare data for plotting (Employment Deprivation)
# For Employment Scatter Plot
data_employment = merged_depression_employment[[
    "Value_depression",  # Depression Prevalence
    "Value_employment",  # Employment Deprivation Score
    "Area Name_employment"  # District Name
]].rename(
    columns={
        "Value_depression": "Depression Prevalence",
        "Value_employment": "Employment Deprivation Score",
        "Area Name_employment": "District Name"
    }
)

# Create dual sliders for Employment Deprivation
x_min_slider_emp = alt.param(
    name='Employment_Deprivation_Min',
    bind=alt.binding_range(
        min=data_employment['Employment Deprivation Score'].min(), 
        max=data_employment['Employment Deprivation Score'].max(), 
        step=0.01
    ),
    value=data_employment['Employment Deprivation Score'].min()  # Default value
)

x_max_slider_emp = alt.param(
    name='Employment_Deprivation_Max',
    bind=alt.binding_range(
        min=data_employment['Employment Deprivation Score'].min(), 
        max=data_employment['Employment Deprivation Score'].max(), 
        step=0.01
    ),
    value=data_employment['Employment Deprivation Score'].max()  # Default value
)

y_min_slider_emp = alt.param(
    name='Depression_Prevalence_Min',
    bind=alt.binding_range(
        min=data_employment['Depression Prevalence'].min(), 
        max=data_employment['Depression Prevalence'].max(), 
        step=1
    ),
    value=data_employment['Depression Prevalence'].min()  # Default value
)

y_max_slider_emp = alt.param(
    name='Depression_Prevalence_Max',
    bind=alt.binding_range(
        min=data_employment['Depression Prevalence'].min(), 
        max=data_employment['Depression Prevalence'].max(), 
        step=1
    ),
    value=data_employment['Depression Prevalence'].max()  # Default value
)

# Base chart for scatter plot and regression line
base_emp = alt.Chart(data_employment).transform_filter(
    f"datum['Employment Deprivation Score'] >= {x_min_slider_emp.name} && " +
    f"datum['Employment Deprivation Score'] <= {x_max_slider_emp.name} && " +
    f"datum['Depression Prevalence'] >= {y_min_slider_emp.name} && " +
    f"datum['Depression Prevalence'] <= {y_max_slider_emp.name}"
).add_params(
    x_min_slider_emp, x_max_slider_emp, y_min_slider_emp, y_max_slider_emp
)

# Scatter plot with filtering
scatter_with_regression_emp = base_emp.mark_circle(size=80, color="green").encode(
    x=alt.X('Employment Deprivation Score:Q', title='Employment Deprivation Score (IMD 2019)', scale=alt.Scale(zero=False)),
    y=alt.Y('Depression Prevalence:Q', title='Depression Prevalence (%)', scale=alt.Scale(zero=False)),
    tooltip=[
        alt.Tooltip('District Name:N', title='District Name'),
        alt.Tooltip('Employment Deprivation Score:Q', title='Employment Deprivation Score (IMD 2019)'),
        alt.Tooltip('Depression Prevalence:Q', title='Depression Prevalence (%)')
    ]
).properties(
    width=1000,
    height=600
)

# Regression line that dynamically adjusts to filtered data
regression_line_emp = base_emp.transform_regression(
    'Employment Deprivation Score', 'Depression Prevalence', method='linear'
).mark_line(color='red').encode(
    x='Employment Deprivation Score:Q',
    y='Depression Prevalence:Q'
)

# Combine scatter plot and regression line
final_chart_emp = (scatter_with_regression_emp + regression_line_emp).properties(
    title={
        "text": "Depression Prevalence (2023) vs Employment Deprivation Score (2019)",
        "subtitle": "Use the sliders to adjust the range and explore the data",
        "fontSize": 20,
        "subtitleFontSize":14,
        "anchor": "start"
    }
).configure_title(
    fontSize=18,
    anchor='start',
    color='black'
)

# Display the chart
final_chart_emp.interactive()


output_file = 'C:\Bruno\LSE\PP434 - Automated Data Visualisation\Project\Chart_4_scatter_with_emp_deprivation.json'
final_chart_emp.save(output_file, format='json')



In [26]:
# Prepare data for plotting (Crime Deprivation)
# For Crime Scatter Plot
data_crime = merged_depression_crime[[
    "Value_depression",  # Depression Prevalence
    "Value_crime",  # Crime Deprivation Score
    "Area Name_crime"  # District Name
]].rename(
    columns={
        "Value_depression": "Depression Prevalence",
        "Value_crime": "Crime Deprivation Score",
        "Area Name_crime": "District Name"
    }
)

# Create dual sliders for Crime Deprivation
x_min_slider_crime = alt.param(
    name='Crime_Deprivation_Min',
    bind=alt.binding_range(
        min=data_crime['Crime Deprivation Score'].min(), 
        max=data_crime['Crime Deprivation Score'].max(), 
        step=0.1
    ),
    value=data_crime['Crime Deprivation Score'].min()  # Default value
)

x_max_slider_crime = alt.param(
    name='Crime_Deprivation_Max',
    bind=alt.binding_range(
        min=data_crime['Crime Deprivation Score'].min(), 
        max=data_crime['Crime Deprivation Score'].max(), 
        step=0.1
    ),
    value=data_crime['Crime Deprivation Score'].max()  # Default value
)

y_min_slider_crime = alt.param(
    name='Depression_Prevalence_Min',
    bind=alt.binding_range(
        min=data_crime['Depression Prevalence'].min(), 
        max=data_crime['Depression Prevalence'].max(), 
        step=1
    ),
    value=data_crime['Depression Prevalence'].min()  # Default value
)

y_max_slider_crime = alt.param(
    name='Depression_Prevalence_Max',
    bind=alt.binding_range(
        min=data_crime['Depression Prevalence'].min(), 
        max=data_crime['Depression Prevalence'].max(), 
        step=1
    ),
    value=data_crime['Depression Prevalence'].max()  # Default value
)

# Base chart for scatter plot and regression line
base_crime = alt.Chart(data_crime).transform_filter(
    f"datum['Crime Deprivation Score'] >= {x_min_slider_crime.name} && " +
    f"datum['Crime Deprivation Score'] <= {x_max_slider_crime.name} && " +
    f"datum['Depression Prevalence'] >= {y_min_slider_crime.name} && " +
    f"datum['Depression Prevalence'] <= {y_max_slider_crime.name}"
).add_params(
    x_min_slider_crime, x_max_slider_crime, y_min_slider_crime, y_max_slider_crime
)

# Scatter plot with filtering
scatter_with_regression_crime = base_crime.mark_circle(size=80, color="purple").encode(
    x=alt.X('Crime Deprivation Score:Q', title='Crime Deprivation Score (IMD 2019)', scale=alt.Scale(zero=False)),
    y=alt.Y('Depression Prevalence:Q', title='Depression Prevalence (%)', scale=alt.Scale(zero=False)),
    tooltip=[
        alt.Tooltip('District Name:N', title='District Name'),
        alt.Tooltip('Crime Deprivation Score:Q', title='Crime Deprivation Score (IMD 2019)'),
        alt.Tooltip('Depression Prevalence:Q', title='Depression Prevalence (%)')
    ]
).properties(
    width=1000,
    height=600
)

# Regression line that dynamically adjusts to filtered data
regression_line_crime = base_crime.transform_regression(
    'Crime Deprivation Score', 'Depression Prevalence', method='linear'
).mark_line(color='red').encode(
    x='Crime Deprivation Score:Q',
    y='Depression Prevalence:Q'
)

# Combine scatter plot and regression line
final_chart_crime = (scatter_with_regression_crime + regression_line_crime).properties(
    title={
        "text": "Depression Prevalence (2023) vs Crime Deprivation Score (2019)",
        "subtitle": "Use the sliders to adjust the range and explore the data",
        "fontSize": 20,
        "subtitleFontSize":14,
        "anchor": "start"
    }
).configure_title(
    fontSize=18,
    anchor='start',
    color='black'
)

# Display the chart
final_chart_crime.interactive()

final_chart_crime.save("Chart_5_scatter_with_crime_deprivation.json")